In [14]:
import pandas as pd # manipulacion y analisis de datos
import os # interactua con el sistema de archivos 
from datetime import datetime # trabajar con fechas y horas

TRUCK = {
    "CAT 789C": [
            
    ]
    "CAT 793D": [
        
    ]
}

class FuelDataETL:
    def __init__(self):
        self.DATA_DIR = os.path.abspath(os.path.join("..", "data-set", "val_data_sensor", "raw_data"))
        # Updated to match actual format in the data
        self.VALID_MODELS = {'CAT - 789C', 'CAT - 793D'}
        self.TRUCK_PREFIXES = {str(i).zfill(3) for i in range(210, 226)} | {str(i).zfill(3) for i in range(230, 244)}
        self.raw_df = pd.DataFrame()
        self.transformed_df = pd.DataFrame()
        
    def extract(self):
        """Cargar todos los archivos de despacho relevantes"""
        file_list = [
            f for f in os.listdir(self.DATA_DIR)
            if f.startswith('despacho') and f.endswith(('.xls', '.xlsx'))
        ]
        dfs = []
        for file in file_list:
            try:
                file_path = os.path.join(self.DATA_DIR, file)
                engine = 'openpyxl' if file.endswith('.xlsx') else None
                
                df = pd.read_excel(
                    file_path,
                    engine=engine,
                    usecols=['Veh', 'Descripcion', 'fin_desp', 'volumCorregido']
                )
                dfs.append(df)
                print(f"✅ {file} cargado correctamente")
            except Exception as e:
                print(f"❌ Error cargando {file}: {str(e)}")
                continue
        if dfs:
            self.raw_df = pd.concat(dfs, ignore_index=True)
            print(f"\n📦 Datos brutos combinados: {len(self.raw_df)} registros")
        else:
            raise ValueError("No se encontraron archivos válidos para procesar")
    
    def transform(self):
        """Aplicar todas las transformaciones necesarias"""
        if self.raw_df.empty:
            raise ValueError("No hay datos para transformar")
        
        # Limpieza inicial
        df = self.raw_df.copy()
        df = df.dropna(subset=['Veh', 'Descripcion', 'fin_desp'])
        
        # Filtrado por descripción
        df['Descripcion'] = df['Descripcion'].str.upper().str.strip()
        valid_desc = {d.upper() for d in self.VALID_MODEL}
        df = df[df['Descripcion'].isin(valid_desc)]
        
        # Procesamiento de Vehículos - Manejo flexible del formato
        # Extraemos hasta 3 dígitos consecutivos donde sea que estén en el campo Veh
        df['Veh_Prefix'] = df['Veh'].astype(str).str.extract(r'(\d{3})')[0]
        df = df[df['Veh_Prefix'].isin(self.TRUCK_PREFIXES)]
        df['Veh'] = 'T-' + df['Veh_Prefix']
        df = df.drop('Veh_Prefix', axis=1)
        
        # Transformación de fechas - No convertir si ya son datetime
        if not pd.api.types.is_datetime64_any_dtype(df['fin_desp']):
            df['fin_desp'] = pd.to_datetime(
                df['fin_desp'],
                errors='coerce',
                dayfirst=True
            )
        
        df = df.dropna(subset=['fin_desp'])
        
        # Crear nuevas columnas temporales
        df = df.assign(
            ShiftDate=df['fin_desp'].dt.normalize(),
            TimeStamp=df['fin_desp'].dt.time
        )
        
        # Renombrar y seleccionar columnas finales
        self.transformed_df = df.rename(columns={
            'Veh': 'Equipment',
            'volumCorregido': 'FuelLevelLiters'
        })[["ShiftDate", "TimeStamp", "Equipment", "FuelLevelLiters"]]
        
        # Validación final
        self._validate_data()
        print(f"\n✨ Transformación completada: {len(self.transformed_df)} registros válidos")
    
    def _validate_data(self):
        """Validación de consistencia de datos"""
        # Verificar rangos de fechas
        min_date = self.transformed_df['ShiftDate'].min() if not self.transformed_df.empty else None
        max_date = self.transformed_df['ShiftDate'].max() if not self.transformed_df.empty else None
        print(f"📅 Rango de fechas: {min_date} - {max_date}")
        
        # Verificar valores únicos de equipos
        unique_equipment = self.transformed_df['Equipment'].unique() if not self.transformed_df.empty else []
        equipment_str = ', '.join(sorted(unique_equipment)) if len(unique_equipment) > 0 else "Ninguno"
        print(f"🚚 Equipos detectados: {equipment_str}")
        
        # Verificar valores de combustible
        fuel_stats = self.transformed_df['FuelLevelLiters'].describe() if not self.transformed_df.empty else None
        print(f"⛽ Estadísticas de combustible:\n{fuel_stats}")

    def load(self, output_path='fuel_data.csv'):
        
        if not self.transformed_df.empty:
            self.transformed_df.to_csv(output_path, index=False)
            print(f"\n💾 Datos guardados en: {output_path}")
            return True
        print("⚠️ No hay datos para guardar")
        return False

    
if __name__ == '__main__':
    # Ejecución del pipeline completo
    etl = FuelDataETL()
    
    etl.extract()
    etl.transform()
    etl.load()

✅ despachos_surtidor_truckshop.xlsx cargado correctamente
✅ despacho_p068.xlsx cargado correctamente
✅ despacho_stt.xlsx cargado correctamente

📦 Datos brutos combinados: 9894 registros
📅 Rango de fechas: 2023-12-31 00:00:00 - 2025-02-28 00:00:00
🚚 Equipos detectados: T-210, T-211, T-212, T-213, T-214, T-215, T-216, T-217, T-218, T-219, T-220, T-221, T-222, T-223, T-224, T-225, T-230, T-231, T-232, T-233, T-234, T-235, T-236, T-237, T-238, T-239, T-240, T-241, T-242, T-243
⛽ Estadísticas de combustible:
count    9894.000000
mean     2486.375884
std       460.777938
min         0.000000
25%      2212.000000
50%      2430.000000
75%      2679.000000
max      4000.000000
Name: FuelLevelLiters, dtype: float64

✨ Transformación completada: 9894 registros válidos

💾 Datos guardados en: fuel_data.csv


In [13]:
# texto = {str(i).zfill(3) for i in range(210, 226)} | {str(i).zfill(3) for i in range(230, 244)}
diccionario = {str(i).zfill(3) for i in range(210, 226)} | {str(i).zfill(3) for i in range(230, 244)}

for i in diccionario:
    print(type(i))


<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-<class 'str'>-

In [51]:
df = etl.load()
print(df.head())



💾 Datos cargados en DataFrame: transformed_fuel_data con 9894 registros
    ShiftDate        TimeStamp Equipment  FuelLevelLiters
4  2023-12-31  04:59:22.187000     T-216             2550
6  2023-12-31  05:24:42.980000     T-210             1473
10 2023-12-31  08:20:41.740000     T-233             1821
34 2024-01-02  04:06:59.627000     T-215             2805
35 2024-01-02  04:22:15.587000     T-221             2177


In [52]:
import pandas as pd
import os
import re
from datetime import datetime

class FuelDataETL:
    def __init__(self):
        self.DATA_DIR = os.path.abspath(os.path.join("..", "data-set", "val_data_sensor", "raw_data"))
        self.valid_equipment = [f"T-{str(i).zfill(3)}" for i in list(range(210, 226)) + list(range(230, 244))]
        self.final_columns = ['ShiftDate', 'TimeStamp', 'Equipment', 'FuelLevelLiters']
        self.transformed_df = pd.DataFrame(columns=self.final_columns)

    def extract_transform(self):
        """Procesa todos los archivos equipo_*.xlsx"""
        equipment_files = [f for f in os.listdir(self.DATA_DIR) if f.startswith('equipo') and f.endswith('.xlsx')]
        
        for file in equipment_files:
            file_path = os.path.join(self.DATA_DIR, file)
            xls = pd.ExcelFile(file_path)
            
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
                self._process_sheet(df, file, sheet_name)
        
        return self.transformed_df

    def _process_sheet(self, df, filename, sheetname):
        """Procesa cada hoja del archivo Excel"""
        date_pattern = re.compile(r'EQUIPO\s+\w+\s+(\d{1,2}/\d{1,2}/\d{4})', re.IGNORECASE)
        
        for row_idx in range(3):  # Buscar en primeras 3 filas
            for col_idx in range(df.shape[1]):
                cell_value = str(df.iloc[row_idx, col_idx])
                date_match = date_pattern.search(cell_value)
                
                if date_match:
                    shift_date = self._parse_date(date_match.group(1))
                    self._process_columns(df, shift_date, col_idx, row_idx)
                    break

    def _parse_date(self, date_str):
        """Convierte la fecha al formato correcto"""
        try:
            return datetime.strptime(date_str, '%d/%m/%Y').strftime('%Y-%m-%d')
        except:
            return None

    def _process_columns(self, df, shift_date, start_col, header_row):
        """Procesa las columnas de datos"""
        if not shift_date:
            return
            
        # Buscar columna CODIGO EQ.
        for col in range(start_col, min(start_col+6, df.shape[1])):
            if "CODIGO EQ." in str(df.iloc[header_row+1, col]):
                code_col = col
                fuel_col = code_col + 2  # Saltar RECEPCION
                break
        else:
            return

        # Procesar filas de datos
        for row in range(header_row+2, df.shape[0]):
            equipment_code = str(df.iloc[row, code_col]).strip()
            
            if not equipment_code.isdigit():
                continue
                
            equipment = f"T-{equipment_code.zfill(3)}"
            
            if equipment in self.valid_equipment:
                fuel = df.iloc[row, fuel_col]
                
                if pd.notna(fuel) and str(fuel).replace('.','',1).isdigit():
                    new_row = {
                        'ShiftDate': shift_date,
                        'TimeStamp': '00:00:00',
                        'Equipment': equipment,
                        'FuelLevelLiters': float(fuel)
                    }
                    
                    self.transformed_df = pd.concat(
                        [self.transformed_df, pd.DataFrame([new_row])],
                        ignore_index=True
                    )

    def load(self, output_file='fuel_data.csv'):
        """Guarda los datos en CSV"""
        if not self.transformed_df.empty:
            self.transformed_df.to_csv(output_file, index=False)
            print(f"Datos guardados en {output_file}")
        else:
            print("No hay datos para guardar")

# Ejecución del proceso
if __name__ == "__main__":
    etl = FuelDataETL()
    etl.extract_transform()
    
    print("\nMuestra de datos transformados:")
    print(etl.transformed_df.head())
    
    etl.load()

C:\Users\JoseFM\AppData\Local\Temp\ipykernel_12792\4102192940.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.transformed_df = pd.concat(



Muestra de datos transformados:
    ShiftDate TimeStamp Equipment  FuelLevelLiters
0  2024-12-28  00:00:00     T-216           2677.0
1  2024-12-28  00:00:00     T-221           2523.0
2  2024-12-28  00:00:00     T-241           2343.0
3  2024-12-28  00:00:00     T-211           2522.0
4  2024-12-28  00:00:00     T-223           2277.0
Datos guardados en fuel_data.csv


In [7]:
import os
import shutil
from openpyxl import load_workbook, Workbook

def eliminar_columnas_y_formato():
    origen = os.path.join("..", "data-set", "val_data_sensor", "raw_data")
    backup = os.path.join("..", "data-set", "val_data_sensor", "backup_data")
    inicio = 2
    paso = 3

    if not os.path.exists(backup):
        os.makedirs(backup)

    for archivo in os.listdir(origen):
        if archivo.startswith('EQUIPO') and archivo.endswith('.xlsx'):
            ruta_archivo = os.path.join(origen, archivo)

            try:
                if os.path.getsize(ruta_archivo) == 0:
                    print(f"⚠️ {archivo} está vacío, se omite.")
                    continue

                # Backup
                shutil.copy(ruta_archivo, os.path.join(backup, archivo))

                # Cargar archivo original
                wb_original = load_workbook(ruta_archivo)
                wb_limpio = Workbook()
                wb_limpio.remove(wb_limpio.active)  # Quitar hoja por defecto

                for hoja_nombre in wb_original.sheetnames:
                    hoja_origen = wb_original[hoja_nombre]
                    hoja_nueva = wb_limpio.create_sheet(title=hoja_nombre)

                    for fila in hoja_origen.iter_rows(values_only=True):
                        hoja_nueva.append(list(fila))  # Solo valores

                    # Eliminar columnas en la hoja nueva
                    total_columnas = hoja_nueva.max_column
                    columnas_a_eliminar = list(range(2, total_columnas + 1, 3))
                    
                    for col in sorted(columnas_a_eliminar, reverse=True):
                        if col <= hoja_nueva.max_column:
                            hoja_nueva.delete_cols(col)

                    print(f"   🧹 Hoja '{hoja_nombre}': columnas eliminadas -> {columnas_a_eliminar}")

                # Guardar archivo limpio y reemplazar
                wb_limpio.save(ruta_archivo)
                print(f"✅ {archivo} formateado y procesado correctamente")

            except Exception as e:
                print(f"❌ Error en {archivo}: {str(e)}")

if __name__ == "__main__":
    print("=== INICIO DEL LIMPIADOR TOTAL ===")
    eliminar_columnas_y_formato()
    print("=== FINALIZADO ===")


=== INICIO DEL LIMPIADOR TOTAL ===
   🧹 Hoja 'ENERO ': columnas eliminadas -> [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 134, 137, 140, 143, 146, 149, 152, 155, 158, 161, 164, 167, 170, 173, 176, 179, 182, 185, 188]
   🧹 Hoja 'FEBRERO ': columnas eliminadas -> [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101, 104, 107, 110, 113, 116, 119, 122, 125, 128, 131, 134, 137, 140, 143, 146, 149, 152, 155, 158, 161, 164, 167, 170, 173]
   🧹 Hoja 'MARZO': columnas eliminadas -> [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98]
✅ EQUIPO 5742 STT 2025.xlsx formateado y procesado correctamente
   🧹 Hoja 'ENERO ': columnas eliminadas -> [2, 5, 8, 11, 14, 17, 20, 23, 26, 29]
   🧹 Hoja 'FEBRERO': col

In [11]:
import os
import shutil
from openpyxl import load_workbook, Workbook

def reestructurar_columnas():
    origen = os.path.join("..", "data-set", "val_data_sensor", "raw_data")
    backup = os.path.join("..", "data-set", "val_data_sensor", "backup_data")

    if not os.path.exists(backup):
        os.makedirs(backup)

    for archivo in os.listdir(origen):
        if archivo.startswith("EQUIPO") and archivo.endswith(".xlsx"):
            ruta_archivo = os.path.join(origen, archivo)

            try:
                if os.path.getsize(ruta_archivo) == 0:
                    print(f"⚠️ {archivo} está vacío. Se omite.")
                    continue

                # Backup
                shutil.copy(ruta_archivo, os.path.join(backup, archivo))

                # Cargar original
                wb_origen = load_workbook(ruta_archivo)
                wb_nuevo = Workbook()
                hoja_destino = wb_nuevo.active
                hoja_destino.title = "Unificado"

                fila_actual = 1  # Control de filas en hoja destino

                for nombre_hoja in wb_origen.sheetnames:
                    ws = wb_origen[nombre_hoja]
                    max_col = ws.max_column
                    max_row = ws.max_row

                    # Recorrer columnas de dos en dos (C-D, E-F, etc.)
                    for col in range(1, max_col + 1, 2):  # 1-based indexing
                        if col + 1 > max_col:
                            break  # si no hay par completo, saltar

                        for fila in range(1, max_row + 1):
                            val1 = ws.cell(row=fila, column=col).value
                            val2 = ws.cell(row=fila, column=col + 1).value

                            # Solo agregamos si hay al menos un valor
                            if val1 is not None or val2 is not None:
                                hoja_destino.cell(row=fila_actual, column=1, value=val1)
                                hoja_destino.cell(row=fila_actual, column=2, value=val2)
                                fila_actual += 1

                # Guardar sobre el archivo original
                wb_nuevo.save(ruta_archivo)
                print(f"✅ {archivo} reestructurado exitosamente")

            except Exception as e:
                print(f"❌ Error en {archivo}: {str(e)}")

if __name__ == "__main__":
    print("=== INICIO REESTRUCTURACIÓN DE COLUMNAS ===")
    reestructurar_columnas()
    print("=== FINALIZADO ===")


=== INICIO REESTRUCTURACIÓN DE COLUMNAS ===
✅ EQUIPO 5742 STT 2025.xlsx reestructurado exitosamente
✅ EQUIPO 5742 STT REVISADO 2024.xlsx reestructurado exitosamente
✅ EQUIPO P68 (Version  1) 2024.xlsx reestructurado exitosamente
✅ EQUIPO P68 (version 1) 2025.xlsx reestructurado exitosamente
=== FINALIZADO ===
